In [2]:
import pandas as pd
import re
import nltk

In [3]:
df = pd.read_csv('train.csv').fillna(' ')
df.head()

,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,"Fine, I don't think blocking is a solution to ...",0,0,0,0,0,0
1,ter. v er.,1,0,1,0,1,0
2,.\n\nNeither am I. Pants suck. They should all...,1,0,1,0,0,0
3,"iYou're missing my point so badly, I can't tel...",0,0,0,0,0,0
4,photo \n\ni'll see what i can do to it. i've...,0,0,0,0,0,0


In [4]:
df_test = pd.read_csv('test.csv').fillna(' ')
df_test.head()

,id,comment_text
0,id0,ok thanks can i request a new article under su...
1,id1,"I deleted Bluesnews.com, which was a redirect ..."
2,id2,"gYes, there is definite value in documenting t..."
3,id3,You STUPID ASSHOLES. You dont know anything!!!...
4,id4,",""\n\nSearch my ass instead. I read the corre..."


In [5]:
X_train = df['comment_text']
X_test = df_test['comment_text']
X = pd.concat([X_train, X_test])

In [103]:
toxic = df[df['toxic'] == 1]['comment_text'].apply(lambda x: tokenize(x))

In [6]:
def tokenize(text):
    comp = re.compile('[A-Za-z\'\-]+')
    tokens = comp.findall(text.lower())
    return tokens

In [116]:
freq = nltk.FreqDist(word for sent in toxic.values for word in sent)

In [128]:
sorted(freq.items(), key=lambda x: x[1])

[('hellx', 1),
 ('dolescum', 1),
 ('couse', 1),
 ('kinghy', 1),
 ('cordial', 1),
 ('downtrodden', 1),
 ('commoner', 1),
 ('wieght', 1),
 ('uniployed', 1),
 ('pogo', 1),
 ('modeled', 1),
 ('oa', 1),
 ('bartlett', 1),
 ('hocrriffic', 1),
 ('wiki-revenge', 1),
 ('vwar', 1),
 ('hottest', 1),
 ('farrakhanistic', 1),
 ('farrakhan', 1),
 ('cibao', 1),
 ('sammy', 1),
 ('sosa', 1),
 ('one-droppism', 1),
 ('hyperdecent', 1),
 ('dominicans', 1),
 ('cibae', 1),
 ('generalize', 1),
 ('nationlaity', 1),
 ('dominican', 1),
 ('intermarrired', 1),
 ('mulattoes', 1),
 ('bad-mouthing', 1),
 ('pea-sized', 1),
 ('neantherthals', 1),
 ('eloping', 1),
 ('rams', 1),
 ('mis-use', 1),
 ('scallywag', 1),
 ('iim', 1),
 ('eah', 1),
 ('clownx', 1),
 ('niggerkite', 1),
 ('oweeds', 1),
 ('bussines', 1),
 ('nanjing', 1),
 ('nazipedia', 1),
 ('explicity', 1),
 ('bodily', 1),
 ('satisfactory', 1),
 ('conflcit', 1),
 ('abuot', 1),
 ('inappropriateness', 1),
 ('michelle', 1),
 ('stpo', 1),
 ('probablay', 1),
 ('bareback',

In [171]:
freq2 = nltk.FreqDist(word for sent in X_test.apply(lambda x: tokenize(x)) for word in sent)

In [175]:
freq.keys() & freq2.keys()

{'safety',
 'nb',
 'sooooo',
 'swartz',
 'happiness',
 'douchae',
 'macamhlaidh',
 'grossly',
 'easily',
 'clarify',
 'path',
 'criteria',
 'grade',
 'feminist',
 'coren',
 'wikip',
 'hometown',
 'winds',
 'dalits',
 'anthony',
 'foad',
 "misterwiki's",
 'association',
 'importance',
 'pun',
 'redknecks',
 'paintings',
 'glass',
 "gc's",
 'admin',
 'btng',
 'reproduced',
 'looked',
 'spare',
 'voiced',
 'version',
 'industrial',
 'gutted',
 'standardized',
 'query',
 'fundamental',
 'toes',
 'gong',
 'res',
 'chalk',
 'kicking',
 'premature',
 'rangers',
 'kirk',
 'poof',
 'hezbollah',
 'addictive',
 'hijacking',
 'handlers',
 'nnot',
 'fewest',
 'guide',
 'prostate',
 'rub',
 'uninvolved',
 'tertiary',
 "din't",
 'ull',
 'cameroon',
 'someones',
 'mongrel',
 'connotation',
 'lobby',
 'klux',
 'lare',
 'barack',
 'texas',
 'slag',
 'glorious',
 'sayerslle',
 'acceptable',
 'have',
 'true',
 'write-up',
 'yuck',
 'editwarring',
 'cambridgebayweather',
 'dissident',
 'pimping',
 'frisco'

In [6]:
X.head()

0    Fine, I don't think blocking is a solution to ...
1                                           ter. v er.
2    .\n\nNeither am I. Pants suck. They should all...
3    iYou're missing my point so badly, I can't tel...
4     photo \n\ni'll see what i can do to it.  i've...
Name: comment_text, dtype: object

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer

In [9]:
from scipy.sparse import hstack
import numpy as np
from sklearn.model_selection import cross_val_score

In [10]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{4,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

In [11]:
word_vectorizer.fit(X)
X_train_word = word_vectorizer.transform(X_train)
X_test_word = word_vectorizer.transform(X_test)

In [12]:
terms = word_vectorizer.get_feature_names()

In [13]:
# show topic descriptors
def get_descriptor( terms, H, topic_index, top ):
    top_indices = np.argsort( H[topic_index,:] )[::-1]
    top_terms = []
    for term_index in top_indices[0:top]:
        top_terms.append( terms[term_index] )
    return top_terms

def get_all_descriptors(k, H, terms, top):
    key_words = []
    for topic_index in range(k):
        descriptor = get_descriptor( terms, H, topic_index, top )
        key_words.append(descriptor)
        str_descriptor = ", ".join( descriptor )
        print("Topic %02d: %s" % ( topic_index+1, str_descriptor ) )
    return key_words

In [14]:
from sklearn.decomposition import NMF

In [20]:
types = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

In [16]:
k = 7

nmf = NMF(init="nndsvd", n_components=k, random_state=42) 
W = nmf.fit_transform(X_train_word) # тематическое представление тектов
H = nmf.components_  # темы с вероятностями слов быть по ним

key_words = get_all_descriptors(k, H, terms, 20)

Topic 01: fuck, shut, asshole, cunt, mother, faggot, fuckin, want, think, dont, nigger, talking, bitches, face, hole, delete, motherfucker, tell, bastard, edit
Topic 02: wikipedia, like, just, article, know, people, think, stupid, stop, time, edit, want, make, good, editing, articles, really, blocked, right, edits
Topic 03: fucking, cunt, faggot, asshole, life, mother, idiot, retard, hell, bastard, hate, block, kill, stupid, moron, fucker, stop, piece, loser, right
Topic 04: suck, dick, cock, faggot, balls, asshole, love, sucks, penis, like, dicks, cunt, lick, head, yeah, likes, dont, pussy, life, nigger
Topic 05: talk, page, thanks, user, contribs, pages, edit, redirect, welcome, editing, thank, message, help, discussion, comments, questions, vandalize, comment, leave, hello
Topic 06: bitch, stupid, little, asshole, shut, fuckin, nigga, motherfucker, cunt, cock, bastard, whore, pussy, kill, gonna, damn, hell, ugly, balls, dont
Topic 07: shit, piece, hell, dont, little, kill, delete, f

In [17]:
key_words = set([word for theme in key_words for word in theme])

In [18]:
len(key_words)

90

In [ ]:
toxic = []
severe_toxic = []
obscene = ['suck', 'dick', 'cock', 'faggot', 'balls', 'asshole', 'sucks', 'penis', 'dicks', 
           'cunt', 'lick', 'yeah', 'pussy', 'nigger'] # topic 4
threat = []
insult = ['fucking', 'cunt', 'faggot', 'asshole', 'mother', 'idiot', 'retard', 'hell', 
          'bastard', 'hate', 'block', 'kill', 'stupid', 'moron', 'fucker', 'stop', 'piece', 'loser', 'right'] # topic 3
identity_hate = ['fuck', 'shut', 'asshole', 'cunt', 'faggot', 'mother', 'fuckin', 'nigger',
                 'bitches', 'hole', 'delete', 'motherfucker', 'bastard'] # topic 1

In [19]:
df_words = pd.DataFrame(columns=key_words)

In [20]:
key_dict = {word: [] for word in key_words}
for word in key_words:
    for message in X_train:
        if word in message:
            key_dict[word].append(1) 
        else:
            key_dict[word].append(0)

In [21]:
for word in key_dict:
    df_words[word] = key_dict[word]

In [36]:
df_words_test = pd.DataFrame(columns=key_words)

In [38]:
key_dict_test = {word: [] for word in key_words}
for word in key_words:
    for message in X_test:
        if word in message:
            key_dict_test[word].append(1) 
        else:
            key_dict_test[word].append(0)

In [39]:
for word in key_dict_test:
    df_words_test[word] = key_dict_test[word]

In [40]:
df_words_test.head()

,edits,sucks,delete,love,fuckin,article,fucker,pussy,people,loser,...,comment,pages,life,piece,message,likes,hate,stupid,talking,user
0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)

In [16]:
char_vectorizer.fit(X)
X_train_char = char_vectorizer.transform(X_train)
X_test_char = char_vectorizer.transform(X_test)

In [17]:
train_features = hstack([X_train_word, X_train_char])
test_features = hstack([X_test_word, X_test_char])

In [18]:
import scipy

In [34]:
# Добавляем в вектора наши дополнительные признаки
X_train_new = scipy.sparse.hstack([train_features, 
    scipy.sparse.csr_matrix(df_words[list(key_words)].values)])

In [41]:
# Добавляем в вектора наши дополнительные признаки
X_test_new = scipy.sparse.hstack([test_features, 
    scipy.sparse.csr_matrix(df_words_test[list(key_words)].values)])

In [35]:
X_train_new

<22404x60090 sparse matrix of type '<class 'numpy.float64'>'
	with 15653290 stored elements in COOrdinate format>

In [42]:
X_test_new

<12064x60090 sparse matrix of type '<class 'numpy.float64'>'
	with 8479329 stored elements in COOrdinate format>

In [1]:
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']}

In [21]:
y_train = df[types]

In [37]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier

In [36]:
# submission = pd.DataFrame.from_dict({'id': df_test['id']})
# scores = []

# y_train = df[typ]
lr = OneVsRestClassifier(LinearSVC(C=0.1))

cv_score = np.mean(cross_val_score(lr, train_features, y_train, cv=3, scoring='roc_auc'))
# scores.append(cv_score)
# print('CV score for class {} is {}'.format(typ, cv_score))

lr.fit(train_features, y_train)
submission = lr.predict_proba(test_features)

AttributeError: 'LinearSVC' object has no attribute 'predict_proba'

In [44]:
submission.head()

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,id0,0.2,0.0,0.1,0.0,0.1,0.0
1,id1,0.1,0.0,0.1,0.0,0.1,0.0
2,id2,0.1,0.0,0.1,0.0,0.1,0.0
3,id3,0.9,0.0,0.5,0.0,0.6,0.1
4,id4,0.1,0.0,0.1,0.0,0.0,0.0


In [45]:
print('Total CV score is {}'.format(np.mean(scores)))

Total CV score is 0.8927121018794498


In [71]:
submission.to_csv('submission2.csv', index=False)

In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, f1_score

In [32]:
gcv = GridSearchCV(estimator=lr, param_grid=params, scoring='f1')
gcv.fit(X_vec, y)

/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/metric

GridSearchCV(cv=None, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='f1', verbose=0)

In [33]:
pd.DataFrame(gcv.cv_results_).sort_values(by='rank_test_score').head()

/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split0_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/socur/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:122: FutureWarning: You are accessing a training score ('split1_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)
/home/socur/anaconda3/lib/python3.6/

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_C,param_penalty,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
13,3.694677,0.005972,0.879253,0.997736,1000,l2,"{'C': 1000, 'penalty': 'l2'}",1,0.877981,0.997358,0.879316,0.997682,0.880461,0.998167,0.071274,0.000085,0.001014,0.000332
11,2.319563,0.006220,0.879115,0.997573,100,l2,"{'C': 100, 'penalty': 'l2'}",2,0.877860,0.997142,0.880111,0.997574,0.879375,0.998005,0.295282,0.000301,0.000937,0.000352
9,1.252314,0.006068,0.876607,0.996169,10,l2,"{'C': 10, 'penalty': 'l2'}",3,0.875108,0.995629,0.877163,0.996062,0.877549,0.996815,0.018323,0.000138,0.001071,0.000490
10,1.743346,0.009271,0.870900,0.997700,100,l1,"{'C': 100, 'penalty': 'l1'}",4,0.870715,0.997304,0.871287,0.997682,0.870697,0.998113,0.148324,0.002377,0.000274,0.000330
12,1.706011,0.008081,0.870135,0.997718,1000,l1,"{'C': 1000, 'penalty': 'l1'}",5,0.866324,0.997358,0.873622,0.997628,0.870460,0.998167,0.132558,0.001534,0.002988,0.000336
